# Predicting Strokes from Admission Data

In this exercise we are going to try to predict strokes from admission data with various models, and learn how to deal with the issue of class imbalance.

Uses the [stroke dataset from Kaggle](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset) along with the `sklearn` and `pandas` packages!

## First Steps
To begin with, let's setup our notebook with the necessary packages as well as grab the data from Kaggle!

In [ ]:
# TODO: Establish venv/conda/package environment
# Commands with `%` run in the command line instead of within python so we don't have to do this within a seperate terminal!
# In this case, we are making sure our environment/colab instance has installed the latest versions of several commonly used data science packages
%pip install pandas numpy matplotlib

In [3]:
# Setup matplotlib to display plots correctly within pandas
%matplotlib inline

# Import packages into our runtime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Download from Kaggle

Those who have downloaded and setup the kaggle CLI (Command Line Interface) can run the following command to download the dataset:

In [9]:
# Download the stroke prediction dataset into the ./data folder
!kaggle datasets download -d fedesoriano/stroke-prediction-dataset --path ./data

100%|███████████████████████████████████████| 67.4k/67.4k [00:00<00:00, 457kB/s]
100%|███████████████████████████████████████| 67.4k/67.4k [00:00<00:00, 455kB/s]


## Data Exploration
Now we have the data downloaded, we need to load this into a dataframe so we can explore it, and perform further analysis.

We can either do this by unzipping the dataset or just loading it directly with pandas:

In [11]:
# No need to unzip the file - we can load the .csv file within directly into a DataFrame (commonly notated as `df`)
df = pd.read_csv("./data/stroke-prediction-dataset.zip")

It is good practice to explore what data we are actually dealing with and get a *feel* for it. We should check for missing data, what data is present, and relevant datatypes!

There are many ways of doing this with `pandas` ([API](https://pandas.pydata.org/docs/reference/frame.html#attributes-and-underlying-data)) - the most useful and commonly used methods being `head()`, `info()`, `describe()` initially:

In [16]:
# Let's have a look at the first 10 entries of the dataset
df.head(10)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
9,60491,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [22]:
# Describe is useful for looking at continuous data (i.e. int or float datatypes) - we can see it also has a look at data which is
# stored as booleans, as well as ID's (i.e. the id, hypertension, heart_disease, and stroke coluns)

# We can see
df.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,4909.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.854067,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.500000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,33.100000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [23]:
# We can exclude the boolean or ID data by selecting certain rows for example

df[["age", "avg_glucose_level", "bmi"]].describe()

,age,avg_glucose_level,bmi
count,5110.000000,5110.000000,4909.000000
mean,43.226614,106.147677,28.893237
std,22.612647,45.283560,7.854067
min,0.080000,55.120000,10.300000
25%,25.000000,77.245000,23.500000
50%,45.000000,91.885000,28.100000
75%,61.000000,114.090000,33.100000
max,82.000000,271.740000,97.600000


In [25]:
# TODO: Produce graphs, correlate etc.

## Building a Model

In [ ]:
# TODO: Build first models
# FIXME: test

## Unbalanced Datasets

In [26]:
# TODO: Explain problem with unbalanced datasets
# TODO: Accuracy paradox --> look at alternative, e.g. F1 score, AUC
# TODO: Explore imbalanced-learn package amongst others